In [ ]:
!pip install mne
!pip install numpy matplotlib pywavelets mne
!pip install numpy mne matplotlib scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 37.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import mne
import matplotlib.pyplot as plt
from scipy.signal import detrend
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import numpy as np
import mne
import matplotlib.pyplot as plt
from scipy.signal import detrend

In [ ]:
path = cleaned_path = "/content/drive/MyDrive/데캡디/데캡디(딥슬립딥드림)공유문서함/DATA/Pre_rev/pre_SN001.edf"
path2 = "/content/drive/MyDrive/데캡디/데캡디(딥슬립딥드림)공유문서함/DATA/sleep_data/recordings/SN001_sleepscoring.txt"
# EDF 파일 로드
raw = mne.io.read_raw_edf(path, preload=True)


Extracting EDF parameters from /content/drive/MyDrive/데캡디/데캡디(딥슬립딥드림)공유문서함/DATA/Pre_rev/pre_SN001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2564999  =      0.000 ... 25649.990 secs...


In [ ]:
channel_types = {
    raw.ch_names[0]: 'eeg',  # 인덱스 0
    raw.ch_names[1]: 'eeg',  # 인덱스 1
    raw.ch_names[2]: 'eeg',  # 인덱스 2
    raw.ch_names[3]: 'eeg',  # 인덱스 3
    raw.ch_names[4]: 'emg',  # 인덱스 4, EMG chin
    raw.ch_names[5]: 'eog',  # 인덱스 5, EOG
    raw.ch_names[6]: 'eog',  # 인덱스 6, EOG
    raw.ch_names[7]: 'ecg'   # 인덱스 7, ECG
}

In [ ]:
# 채널 타입 설정
raw.set_channel_types(channel_types)


Measurement date,"May 14, 2024 04:40:13 GMT"
Experimenter,Unknown
Participant,X
Digitized points,Not available
Good channels,"4 EEG, 1 EMG, 2 EOG, 1 ECG"
Bad channels,None
EOG channels,"EOG E1-M2, EOG E2-M2"
ECG channels,ECG
Sampling frequency,100.00 Hz
Highpass,0.00 Hz
Lowpass,50.00 Hz


In [ ]:
raw.rename_channels({'EEG F4-M1':'F4'})
raw.rename_channels({'EEG C4-M1':'C4'})
raw.rename_channels({'EEG O2-M1':'O2'})
raw.rename_channels({'EEG C3-M2':'C3'})

Measurement date,"May 14, 2024 04:40:13 GMT"
Experimenter,Unknown
Participant,X
Digitized points,Not available
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,100.00 Hz
Highpass,0.00 Hz
Lowpass,50.00 Hz


In [ ]:
eeg = raw.copy().pick_channels(['O2'])

actual_O_signal = eeg.get_data()[0]  # EMG 데이터 배열

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


In [ ]:
# 수면 단계 데이터 로드 (30초 단위로 기록된 수면 단계)
with open(path2, 'r') as file:
    lines = file.readlines()[1:]  # 첫 번째 행을 건너뛰고 나머지를 읽음

# 'Lights off'와 'Lights on' 항목을 제외한 수면 단계 추출
sleep_stages = []
for line in lines:
    if 'Lights off' not in line and 'Lights on' not in line:
        stage = line.split(',')[4].strip()
        sleep_stages.append(stage)

# 샘플링 레이트 설정
new_sampling_rate = 100

# RMS 계산 함수
def calculate_rms(signal):
    return np.sqrt(np.mean(signal**2))

# 전체 EMG 신호에 대한 RMS 값 계산
total_rms = calculate_rms(actual_emg_signal)

# R 단계로 매핑된 RMS 값 계산
stage_duration_sec = 30  # 각 수면 단계의 지속 시간 (초)
stage_sample_count = stage_duration_sec * new_sampling_rate

# R 단계의 RMS 값 계산
rms_r_stage_values = []
for i, stage in enumerate(sleep_stages):
    if stage == 'Sleep stage R':
        start_idx = i * stage_sample_count
        end_idx = (i + 1) * stage_sample_count
        if end_idx <= len(actual_emg_signal):
            r_stage_signal = actual_emg_signal[start_idx:end_idx]
            rms_r_stage_values.append(calculate_rms(r_stage_signal))

# R 단계 RMS 값의 평균 계산
average_r_stage_rms = np.mean(rms_r_stage_values)

print(f"전체 데이터에 대한 RMS 값: {total_rms}")
print(f"R 단계에 대한 RMS 값의 평균: {average_r_stage_rms}")

전체 데이터에 대한 RMS 값: 1.755598120201873e-11
R 단계에 대한 RMS 값의 평균: 1.7569255043229868e-11
